<a href="https://colab.research.google.com/github/allobo49/Homework1_Finance/blob/main/Homework1_Finance_New.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

# Data Sample Selection & Cleaning

In [ ]:
static_carbon = pd.read_excel("https://raw.githubusercontent.com/allobo49/Homework1_Finance/main/Files/Static_Carbon.xlsx")
mv_usd_m = pd.read_excel("https://raw.githubusercontent.com/allobo49/Homework1_Finance/main/Files/DS_MV_USD_M.xlsx")
mv_usd_y = pd.read_excel("https://raw.githubusercontent.com/allobo49/Homework1_Finance/main/Files/DS_MV_USD_Y.xlsx")
ri_usd_m = pd.read_excel("https://raw.githubusercontent.com/allobo49/Homework1_Finance/main/Files/DS_RI_T_USD_M.xlsx")
ri_usd_y = pd.read_excel("https://raw.githubusercontent.com/allobo49/Homework1_Finance/main/Files/DS_RI_T_USD_Y.xlsx")
revenue = pd.read_excel("https://raw.githubusercontent.com/allobo49/Homework1_Finance/main/Files/TC_Revenue.xlsx")
scope1 = pd.read_excel("https://raw.githubusercontent.com/allobo49/Homework1_Finance/main/Files/TC_Scope1.xlsx")
scope1_intensity = pd.read_excel("https://raw.githubusercontent.com/allobo49/Homework1_Finance/main/Files/TC_Scope1Intensity.xlsx")
scope2 = pd.read_excel("https://raw.githubusercontent.com/allobo49/Homework1_Finance/main/Files/TC_Scope2.xlsx")
scope2_intensity = pd.read_excel("https://raw.githubusercontent.com/allobo49/Homework1_Finance/main/Files/TC_Scope2Intensity.xlsx")
scope3 = pd.read_excel("https://raw.githubusercontent.com/allobo49/Homework1_Finance/main/Files/TC_Scope3.xlsx")
scope3_intensity = pd.read_excel("https://raw.githubusercontent.com/allobo49/Homework1_Finance/main/Files/TC_Scope3Intensity.xlsx")

In [ ]:
# Define a function to transform the dataframe
def transform_dataframe(df, id_vars, var_name, value_name, has_x_prefix=False):
    # Melt the dataframe to convert the year columns into rows
    df_melted = df.melt(id_vars=id_vars, var_name=var_name, value_name=value_name)
    df_melted[var_name] = df_melted[var_name].astype('string')

    # Remove the 'x' prefix from the year column if it's present
    if has_x_prefix:
        df_melted[var_name] = df_melted[var_name].str.lstrip('x')

    return df_melted

# Dictionary mapping the dataframe variable names to their corresponding new value names
dataframes_info = {
    'mv_usd_y': ('market_value_yearly', True),
    'ri_usd_y': ('return_index_yearly', True),
    'scope1': ('scope1', False),
    'scope1_intensity': ('scope1_intensity', False),
    'scope2': ('scope2', False),
    'scope2_intensity': ('scope2_intensity', False),
    'scope3': ('scope3', False),
    'scope3_intensity': ('scope3_intensity', False),
    'revenue': ('revenue', False)
}

# Loop through the dictionary and apply the transformation function
for df_name, (value_name, has_x_prefix) in dataframes_info.items():
    # Access the actual DataFrame object using df_name as the key in the globals() dictionary
    df_object = globals()[df_name]

    # Apply the transformation function and assign the result to a dynamically named variable
    globals()[df_name] = transform_dataframe(
        df=df_object,  # Pass the actual DataFrame object
        id_vars=['ISIN', 'NAME'],
        var_name='year',
        value_name=value_name,
        has_x_prefix=has_x_prefix
    )

In [ ]:
# List of dataframes to merge
dataframes_to_merge = [mv_usd_y, ri_usd_y, scope1, scope1_intensity, scope2, scope2_intensity, scope3_intensity, revenue]

# Initialize merged dataframe with the first dataframe in the list
merged_df = dataframes_to_merge[0]

# Loop through the rest of the dataframes and merge them one by one
for df in dataframes_to_merge[1:]:
    merged_df = pd.merge(merged_df, df, on=['ISIN', 'NAME', 'year'], how='outer')

merged_df.head()

,ISIN,NAME,year,market_value_yearly,return_index_yearly,scope1,scope1_intensity,scope2,scope2_intensity,scope3_intensity,revenue
0,AN8068571086,SCHLUMBERGER,1999,30859.99,1708.01,NaN,NaN,NaN,NaN,NaN,NaN
1,AN8068571086,SCHLUMBERGER,1999,30859.99,1708.01,NaN,NaN,NaN,NaN,NaN,NaN
2,AN8068571086,SCHLUMBERGER,1999,30859.99,1708.01,NaN,NaN,NaN,NaN,NaN,NaN
3,AN8068571086,SCHLUMBERGER,1999,30859.99,1708.01,NaN,NaN,NaN,NaN,NaN,NaN
4,ARALUA010258,ALUAR,1999,1129.31,1547.04,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# Filter for companies whose region is "EM"
em_companies = static_carbon[static_carbon['Region'] == 'EM']

# Merge based on ISIN, keeping "Company" instead of "NAME", adding "GICSSectorName" and "Country", and dropping "Region"
final_merged_df = pd.merge(merged_df.drop(columns=['NAME']), em_companies.drop(columns=['Region']), on='ISIN', how='left')

# Rename 'Company' to 'NAME' if needed
final_merged_df.rename(columns={'Company': 'NAME'}, inplace=True)

# Reorder the columns of final_merged
column_order = [
    'ISIN', 'NAME', 'Country', 'GICSSectorName', 'year',
    'market_value_yearly', 'return_index_yearly', 'scope1', 'scope1_intensity',
    'scope2', 'scope2_intensity', 'scope3_intensity', 'revenue'
]
final_merged_df = final_merged_df[column_order]

# Keep companies from "EM" region only
final_merged_filtered = final_merged_df[final_merged_df['ISIN'].isin(em_companies['ISIN'])]

final_merged_filtered

,ISIN,NAME,Country,GICSSectorName,year,market_value_yearly,return_index_yearly,scope1,scope1_intensity,scope2,scope2_intensity,scope3_intensity,revenue
4,ARALUA010258,Aluar Aluminio Argentino S.A.I.C.,ARGENTINA,Materials,1999,1129.31,1547.04,NaN,NaN,NaN,NaN,NaN,NaN
5,ARALUA010258,Aluar Aluminio Argentino S.A.I.C.,ARGENTINA,Materials,1999,1129.31,1547.04,NaN,NaN,NaN,NaN,NaN,NaN
6,ARALUA010258,Aluar Aluminio Argentino S.A.I.C.,ARGENTINA,Materials,1999,1129.31,1547.04,NaN,NaN,NaN,NaN,NaN,NaN
7,ARALUA010258,Aluar Aluminio Argentino S.A.I.C.,ARGENTINA,Materials,1999,1129.31,1547.04,NaN,NaN,NaN,NaN,NaN,NaN
8,ARP125991090,Banco BBVA Argentina S.A.,ARGENTINA,Financials,1999,1656.30,418.23,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
243995,ZAE000191342,Altron Limited,SOUTH AFRICA,Information Technology,2021,NaN,NaN,5834.000,12.915,10592.000,23.448,44.102,451.725010
243996,ZAE000255915,Absa Group Limited,SOUTH AFRICA,Financials,2021,NaN,NaN,13649.232,2.607,163149.067,31.159,12.719,2801.712363
243997,ZAE000298253,Northam Platinum Holdings Limited,SOUTH AFRICA,Materials,2021,NaN,NaN,61015.567,28.846,1279042.000,604.692,187.696,2115.197673
243998,ZAE000302618,Aveng Limited,SOUTH AFRICA,Industrials,2021,NaN,NaN,22875.000,16.403,20606.000,14.776,219.446,1394.554470


In [ ]:
#final_merged_filtered_2020 = final_merged_filtered[final_merged_filtered['year'] == '2020']
#final_merged_filtered_2020

,ISIN,NAME,Country,GICSSectorName,year,market_value_yearly,return_index_yearly,scope1,scope1_intensity,scope2,scope2_intensity,scope3_intensity,revenue
172288,ARALUA010258,Aluar Aluminio Argentino S.A.I.C.,ARGENTINA,Materials,2020,1580.34,2861.62,NaN,NaN,NaN,NaN,NaN,NaN
172289,ARALUA010258,Aluar Aluminio Argentino S.A.I.C.,ARGENTINA,Materials,2020,1580.34,2861.62,NaN,NaN,NaN,NaN,NaN,NaN
172290,ARALUA010258,Aluar Aluminio Argentino S.A.I.C.,ARGENTINA,Materials,2020,1580.34,2861.62,NaN,NaN,NaN,NaN,NaN,NaN
172291,ARALUA010258,Aluar Aluminio Argentino S.A.I.C.,ARGENTINA,Materials,2020,1580.34,2861.62,NaN,NaN,NaN,NaN,NaN,NaN
172292,ARP125991090,Banco BBVA Argentina S.A.,ARGENTINA,Financials,2020,1120.45,191.83,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
241947,ZAE000191342,Altron Limited,SOUTH AFRICA,Information Technology,2020,NaN,NaN,7834.000,6.792,6370.876,5.523,44.998,1153.467677
241948,ZAE000255915,Absa Group Limited,SOUTH AFRICA,Financials,2020,NaN,NaN,13698.168,3.693,163153.451,43.985,22.610,3709.318994
241949,ZAE000298253,Northam Platinum Holdings Limited,SOUTH AFRICA,Materials,2020,NaN,NaN,56179.000,49.343,1021183.000,896.915,179.354,1138.550977
241950,ZAE000302618,Aveng Limited,SOUTH AFRICA,Industrials,2020,NaN,NaN,14303.000,15.775,25740.000,28.388,223.549,906.713079


In [ ]:
#number_of_unique_names = final_merged_filtered_2020['NAME'].nunique()
#print(number_of_unique_names)

507
